In [0]:
import requests
from datetime import date


In [0]:
base_url="https://npiregistry.cms.hhs.gov/api/"
params = {
    "version":"2.1",
    "state":"CA",
    "city":"Los Angeles",
    "limit":20
}

In [0]:
response=requests.get(base_url,params)

In [0]:
response=response.json()

In [0]:
npi_list=[result["number"] for result in response.get("results",[])]


In [0]:
current_date=date.today()

In [0]:
for npi in npi_list:
    detailed_results=[]
    new_param={
        "version":"2.1",
        "number": npi,
    }
    response=requests.get(base_url,new_param)
    response=response.json()
    details=response.get("results",[])
    for detail in details:
        npi_number=detail.get("number","")
        basic_info=detail.get("basic",{})
        if detail.get("enumeration_type","")=="NPI-1":
            fname=basic_info.get("first_name","")
            lname=basic_info.get("last_name","")
        else:
            fname=basic_info.get("authorized_official_first_name","")
            lname=basic_info.get("authorized_official_last_name","")
        position=(
            basic_info.get("authorized_official_title_or_position") 
            if "authorized_official_title_or_position" in 
            basic_info.get("authorized_official_title_or_position")
            else "")
        organisation=basic_info.get("organization_name","")
        last_updated=basic_info.get("last_updated","")
        detailed_results.append(
                        {
                            "npi_id": npi_number,
                            "first_name": fname,
                            "last_name": lname,
                            "position": position,
                            "organisation_name": organisation,
                            "last_updated": last_updated,
                            "refreshed_at": current_date,
                        }
                    )
        
    

In [0]:
if detailed_results:
    print(detailed_results)
    df=spark.createDataFrame(detailed_results)
    df.write.format("parquet").mode("overwrite").save("/mnt/bronze/npi_codes")